In [30]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from IPython.display import display

from pkdb_analysis import PKData, PKFilter
from pkdb_analysis.tests.constants import TEST_HDF5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Filter data
A recurring task is to filter data for a certain question. E.g. to compare two groups, or get the subset of data for all healthy smokers.

We work again with our test data set and will filter various subsets from it.

In [5]:
test_data = PKData.from_hdf5(TEST_HDF5)
print(test_data)

------------------------------
PKData (140514028076504)
------------------------------
studies             4 
groups              8  (   86)
individuals       246  ( 3144)
interventions      18  (   18)
outputs          1064  ( 1106)
timecourses        40  (   46)
------------------------------


## Filter functions
The main principle for filtering `PKData` is by using the `*_filter` and `*_exclude` functionality.

A key principle are hereby filter functions which for a given DataFrame return a logical index.

Depending on which subset of information this should be applied the `groups`, `individuals`, `subjects` (groups and individuals), `outputs` and `timecourses`.

## Filter by `study_sid`
A first example is filtering by `study_sid`, i.e. we only want the subset of data from a single study.
An overview over the existing study sids in the dataset is available via

In [6]:
test_data.study_sids

{'PKDB99996', 'PKDB99997', 'PKDB99998', 'PKDB99999'}

Filtering a subset of data works by providing filter/selection functions which select a subset of the data.
The filters are written on the `groups`, `individuals`

In [18]:
def is_PKDB99999(d):
    """Filter for specific study_sid. """
    return d.study_sid == "PKDB99999"

data = test_data.intervention_filter(is_PKDB99999)
print(data)

Concise DataFrames


------------------------------
PKData (140514014462976)
------------------------------
studies             1 
groups              1  (    6)
individuals         6  (   42)
interventions       3  (    3)
outputs           194  (  194)
timecourses         4  (    4)
------------------------------


The PKData now only contains data for the given study_sid:

In [19]:
print(data.study_sids)

{'PKDB99999'}


In [21]:
# for instance interventions
display(data.interventions)

,study_sid,study_name,intervention_pk,raw_pk,normed,name,route,form,application,time,...,substance,value,mean,median,min,max,sd,se,cv,unit
8,PKDB99999,Test1,93,91,True,po75,oral,tablet,single dose,0.0,...,midazolam,0.007500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram
9,PKDB99999,Test1,94,92,True,po15,oral,tablet,single dose,0.0,...,midazolam,0.015000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram
11,PKDB99999,Test1,96,95,True,iv,iv,solution,single dose,0.0,...,midazolam,0.000075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram / kilogram


One could also define this as a simple lambda function

In [24]:
data = test_data.intervention_filter(lambda d: d.study_sid == "PKDB99999")
print(data)

Concise DataFrames


------------------------------
PKData (140514014371120)
------------------------------
studies             1 
groups              1  (    6)
individuals         6  (   42)
interventions       3  (    3)
outputs           194  (  194)
timecourses         4  (    4)
------------------------------


## Concise data
All operations on `PKData` leave the data in a consistent manner. 
E.g. if an intervention is filtered out also all the outputs using this intervention are filtered out.
This behavior is controlled by the `concise` flag on most operations.

In [25]:
t1 = test_data.intervention_filter(is_PKDB99999)
t2 = test_data.intervention_filter(is_PKDB99999, concise=False)
print(t1)
print(t2)

Concise DataFrames


------------------------------
PKData (140514004646768)
------------------------------
studies             1 
groups              1  (    6)
individuals         6  (   42)
interventions       3  (    3)
outputs           194  (  194)
timecourses         4  (    4)
------------------------------
------------------------------
PKData (140514002350032)
------------------------------
studies             4 
groups              8  (   86)
individuals       246  ( 3144)
interventions       6  (    6)
outputs          1064  ( 1106)
timecourses        40  (   46)
------------------------------


In [26]:
# FIXME: only normed data
t1.interventions_mi

,study_sid,study_name,raw_pk,normed,name,route,form,application,time,time_unit,...,substance,value,mean,median,min,max,sd,se,cv,unit
intervention_pk,,,,,,,,,,,,,,,,,,,,,
93,PKDB99999,Test1,91,True,po75,oral,tablet,single dose,0.0,hr,...,midazolam,0.007500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram
94,PKDB99999,Test1,92,True,po15,oral,tablet,single dose,0.0,hr,...,midazolam,0.015000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram
96,PKDB99999,Test1,95,True,iv,iv,solution,single dose,0.0,hr,...,midazolam,0.000075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram / kilogram


In [11]:
t2.interventions_mi

,study_sid,study_name,raw_pk,normed,name,route,form,application,time,time_unit,...,substance,value,mean,median,min,max,sd,se,cv,unit
intervention_pk,,,,,,,,,,,,,,,,,,,,,
93,PKDB99999,Test1,91,True,po75,oral,tablet,single dose,0.0,hr,...,midazolam,0.007500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram
94,PKDB99999,Test1,92,True,po15,oral,tablet,single dose,0.0,hr,...,midazolam,0.015000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram
96,PKDB99999,Test1,95,True,iv,iv,solution,single dose,0.0,hr,...,midazolam,0.000075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gram / kilogram


In [14]:
t2.outputs

,study_sid,study_name,output_pk,intervention_pk,group_pk,individual_pk,normed,calculated,tissue,time,...,substance,value,mean,median,min,max,sd,se,cv,unit
233,PKDB99999,Test1,1417,94,-1,39,False,False,plasma,NaN,...,midazolam,2.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hr
234,PKDB99999,Test1,1426,96,-1,40,False,False,plasma,NaN,...,midazolam,2.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hr
235,PKDB99999,Test1,1427,96,-1,40,False,False,plasma,NaN,...,midazolam,0.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,l/kg
236,PKDB99999,Test1,1428,96,-1,40,False,False,plasma,NaN,...,midazolam,324.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ml/min
237,PKDB99999,Test1,1429,96,-1,40,False,False,plasma,NaN,...,midazolam,4050.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ml/min
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,PKDB99999,Test1,1576,96,20,-1,True,False,plasma,NaN,...,midazolam,NaN,2.400000,NaN,NaN,NaN,0.8,0.327,0.333,hour
423,PKDB99999,Test1,1584,94,20,-1,True,False,plasma,NaN,...,midazolam,NaN,2.100000,NaN,NaN,NaN,0.5,0.204,0.238,hour
424,PKDB99999,Test1,1598,96,20,-1,True,True,plasma,NaN,...,midazolam,NaN,0.000185,NaN,NaN,NaN,NaN,NaN,NaN,gram / liter
425,PKDB99999,Test1,1601,96,20,-1,True,True,plasma,NaN,...,midazolam,NaN,0.650000,NaN,NaN,NaN,NaN,NaN,NaN,liter / kilogram


## Query groups and individuals
### 2.1 Get data for groups with characteristica/keywords X
healthy=True, smoking=N, disease=None,
individual queries and combinations.


In [41]:
def is_healthy(d): 
    # healthy is reported and True
    return (d.measurement_type == "healthy") & (d.choice == "Y")

def disease(d):
    # any disease is reported
    return  d.measurement_type == "disease"

def smoking(d):
    # smoking status is curated for study (this could by Y/N/NR)
    return  d.measurement_type == "smoking"

def nonsmoker(d):
    # smoking is reported and no
    return smoking(d) & (d.choice == "N")

def smoker(d):
    # smoking is reported and yes
    return smoking(d) & (d.choice == "Y")

In [42]:
test_data = PKData.from_hdf5(TEST_HDF5)

`f_idx` can be a single function, or a list of functions. A list of functions are applied successively and is equivalent to "AND logic". "OR logic" can be directly applied on the index.

In [45]:
healthy_nonsmoker = test_data.subject_filter(f_idx=[is_healthy, nonsmoker])
print(healthy_nonsmoker)
healthy_nonsmoker.groups_mi

Concise DataFrames


------------------------------
PKData (140513998511352)
------------------------------
studies             4 
groups              6  (   73)
individuals       246  ( 3144)
interventions       9  (    9)
outputs          1064  ( 1106)
timecourses        40  (   46)
------------------------------


study_sid study_name group_name  group_count  \
group_pk characteristica_pk                                                 
20       481                 PKDB99999      Test1        all            6   
         482                 PKDB99999      Test1        all            6   
         483                 PKDB99999      Test1        all            6   
         484                 PKDB99999      Test1        all            6   
         485                 PKDB99999      Test1        all            6   
...                                ...        ...        ...          ...   
27       1086                PKDB99996      Test4        all           16   
         1087                PKDB99996      Test4        all           16   
         1088                PKDB99996      Test4        all           16   
         1089                PKDB99996      Test4        all           16   
         1090                PKDB99996      Test4        all           16   

                             group_parent_pk  count measurement_type  \
group_pk characteristica_pk                                            
20       481                              -1      6          species   
         482                              -1      6          healthy   
         483                              -1      1          smoking   
         484                              -1      5          smoking   
         485                              -1      6              age   
...                                      ...    ...              ...   
27       1086                             -1     16          alcohol   
         1087                             -1     16           weight   
         1088                             -1     16              age   
         1089                             -1     16        ethnicity   
         1090                             -1     16   overnight fast   

                                   choice substance  value  mean  median  \
group_pk characteristica_pk                                                
20       481                 homo sapiens      None    NaN   NaN     NaN   
         482                            Y      None    NaN   NaN     NaN   
         483                            Y      None    NaN   NaN     NaN   
         484                            N      None    NaN   NaN     NaN   
         485                         None      None    NaN   NaN     NaN   
...                                   ...       ...    ...   ...     ...   
27       1086                           N      None    NaN   NaN     NaN   
         1087                        None      None    NaN  76.7     NaN   
         1088                        None      None    NaN  27.1     NaN   
         1089                          NR      None    NaN   NaN     NaN   
         1090                           Y      None    NaN   NaN     NaN   

                              min   max   sd  se  cv      unit  
group_pk characteristica_pk                                     
20       481                  NaN   NaN  NaN NaN NaN      None  
         482                  NaN   NaN  NaN NaN NaN      None  
         483                  NaN   NaN  NaN NaN NaN      None  
         484                  NaN   NaN  NaN NaN NaN      None  
         485                 25.0  37.0  NaN NaN NaN        yr  
...                           ...   ...  ...  ..  ..       ...  
27       1086                 NaN   NaN  NaN NaN NaN      None  
         1087                 NaN   NaN  6.8 NaN NaN  kilogram  
         1088                 NaN   NaN  3.1 NaN NaN        yr  
         1089                 NaN   NaN  NaN NaN NaN      None  
         1090                 NaN   NaN  NaN NaN NaN      None  

[73 rows x 18 columns]

Often attributes are mixed for groups so we have to exclude the opposites.
In the example, the group `20` consists of 5 smokers and 1 nonsmoker. So for a subset of the group smoking is No.
We can exclude groups via

In [50]:
healthy_nonsmoker = test_data.subject_filter([is_healthy, nonsmoker]).subject_exclude([smoker])
print(healthy_nonsmoker)
display(healthy_nonsmoker.groups_mi)

Concise DataFrames
Concise DataFrames


------------------------------
PKData (140513997558616)
------------------------------
studies             2 
groups              1  (   11)
individuals         2  (    8)
interventions       5  (    5)
outputs           191  (  233)
timecourses        12  (   18)
------------------------------


study_sid study_name group_name  group_count  \
group_pk characteristica_pk                                                 
27       1080                PKDB99996      Test4        all           16   
         1081                PKDB99996      Test4        all           16   
         1082                PKDB99996      Test4        all           16   
         1083                PKDB99996      Test4        all           16   
         1084                PKDB99996      Test4        all           16   
         1085                PKDB99996      Test4        all           16   
         1086                PKDB99996      Test4        all           16   
         1087                PKDB99996      Test4        all           16   
         1088                PKDB99996      Test4        all           16   
         1089                PKDB99996      Test4        all           16   
         1090                PKDB99996      Test4        all           16   

                             group_parent_pk  count measurement_type  \
group_pk characteristica_pk                                            
27       1080                             -1     16          species   
         1081                             -1     16          healthy   
         1082                             -1     16              sex   
         1083                             -1     16          smoking   
         1084                             -1     16       abstinence   
         1085                             -1     16       medication   
         1086                             -1     16          alcohol   
         1087                             -1     16           weight   
         1088                             -1     16              age   
         1089                             -1     16        ethnicity   
         1090                             -1     16   overnight fast   

                                   choice       substance  value  mean  \
group_pk characteristica_pk                                              
27       1080                homo sapiens            None    NaN   NaN   
         1081                           Y            None    NaN   NaN   
         1082                           M            None    NaN   NaN   
         1083                           N            None    NaN   NaN   
         1084                        None  methylxanthine    NaN   NaN   
         1085                           N            None    NaN   NaN   
         1086                           N            None    NaN   NaN   
         1087                        None            None    NaN  76.7   
         1088                        None            None    NaN  27.1   
         1089                          NR            None    NaN   NaN   
         1090                           Y            None    NaN   NaN   

                             median  min  max   sd  se  cv      unit  
group_pk characteristica_pk                                           
27       1080                   NaN  NaN  NaN  NaN NaN NaN      None  
         1081                   NaN  NaN  NaN  NaN NaN NaN      None  
         1082                   NaN  NaN  NaN  NaN NaN NaN      None  
         1083                   NaN  NaN  NaN  NaN NaN NaN      None  
         1084                   NaN  NaN  NaN  NaN NaN NaN      None  
         1085                   NaN  NaN  NaN  NaN NaN NaN      None  
         1086                   NaN  NaN  NaN  NaN NaN NaN      None  
         1087                   NaN  NaN  NaN  6.8 NaN NaN  kilogram  
         1088                   NaN  NaN  NaN  3.1 NaN NaN        yr  
         1089                   NaN  NaN  NaN  NaN NaN NaN      None  
         1090                   NaN  NaN  NaN  NaN NaN NaN      None

In addition often combinations of attributes have to be used to find the correct subjects.
For instance a combination of `healthy` and reported `disease`

In [51]:
def is_healthy(d): 
    # healthy is reported and True
    return (d.measurement_type == "healthy") & (d.choice == "Y")

def disease(d):
    # any disease is reported
    return  d.measurement_type == "disease"

healthy1 = test_data.subject_filter(is_healthy)
healthy2 = test_data.subject_exclude(disease)
healthy3 = test_data.subject_filter(is_healthy).subject_exclude(disease)

print(healthy1)
print(healthy2)
print(healthy3)

Concise DataFrames
Concise DataFrames
Concise DataFrames
Concise DataFrames


------------------------------
PKData (140514011153688)
------------------------------
studies             4 
groups              6  (   73)
individuals       246  ( 3144)
interventions       9  (    9)
outputs          1064  ( 1106)
timecourses        40  (   46)
------------------------------
------------------------------
PKData (140514007956560)
------------------------------
studies             4 
groups              6  (   73)
individuals       246  ( 3144)
interventions       9  (    9)
outputs          1064  ( 1106)
timecourses        40  (   46)
------------------------------
------------------------------
PKData (140514014190056)
------------------------------
studies             4 
groups              6  (   73)
individuals       246  ( 3144)
interventions       9  (    9)
outputs          1064  ( 1106)
timecourses        40  (   46)
------------------------------


## 3 Query interventions
### 3.1 Get outputs/timecourses for intervention with substance
intervention with measurement_type "dosing" and substance "caffeine"

In [54]:
def dosing_and_caffeine(d):
    return ((d["measurement_type"]=="dosing") & (d["substance"]=="caffeine"))

### 3.2 Get outputs/timecourses where multiple interventions were given

In [55]:
test_data = PKData.from_hdf5(TEST_HDF5)

In [56]:
caffeine_data = test_data.intervention_filter(dosing_and_caffeine)

Concise DataFrames


In [57]:
print(caffeine_data)

------------------------------
PKData (140513995640616)
------------------------------
studies             1 
groups              1  (   11)
individuals         0  (    0)
interventions       1  (    1)
outputs            71  (   71)
timecourses        12  (   12)
------------------------------


## 4 Query outputs/timecourses
### 4.1 query by measurement_type
filter all outputs with measurement_type auc_inf

In [60]:
def is_auc_inf(d):
    return (d["measurement_type"]=="auc_inf")  

test_data = PKData.from_hdf5(TEST_HDF5)

# keep_timecourses removes timecourses. 
test_data = test_data.output_filter(is_auc_inf).timecourses_delete()
print(test_data)

Concise DataFrames
Concise DataFrames


------------------------------
PKData (140514030333680)
------------------------------
studies             3 
groups              6  (   73)
individuals       118  ( 1534)
interventions       5  (    5)
outputs           276  (  278)
timecourses         0  (    0)
------------------------------


## 5 Other Query others
### 5.1 Complex
get clearance of codeine for all.h5 subjects, which have been phenotyped for cyp2d6. 


In [61]:
def is_cyp2d6_phenotyped(d):
    cyp2d6_phenotype_substances = ['spar/(2hspar+5hspar)', 'deb/4hdeb', 'dtf/dmt']
    return d["measurement_type"].isin(["metabolic phenotype", "metabolic ratio"]) & d["substance"].isin(cyp2d6_phenotype_substances)

def codeine_clearance(d):
    return (d["measurement_type"]=="clearance") & (d["substance"]=="codeine")                                                        

In [62]:
test_data = PKData.from_hdf5(TEST_HDF5)

In [64]:
phenotyped_data = test_data.output_filter(is_cyp2d6_phenotyped)

Concise DataFrames


In [67]:
test_data.groups = phenotyped_data.groups
test_data.individuals = phenotyped_data.individuals
test_data = test_data.output_filter(codeine_clearance).timecourses_delete()

Concise DataFrames
Concise DataFrames


In [68]:
print(test_data)

------------------------------
PKData (140513999434248)
------------------------------
studies             0 
groups              0  (    0)
individuals         0  (    0)
interventions       0  (    0)
outputs             0  (    0)
timecourses         0  (    0)
------------------------------


## 6  Pitfalls 

In [71]:
test_data = PKData.from_hdf5(TEST_HDF5)
# Wrong 
def is_healthy_smoker(d): 
    """ This will yield zero subjects. No characteristica satisfy measurement_type == 'healthy' and measurement_type == 'disease'. 
    """
    return ((d["measurement_type"]=="healthy") & (d["choice"]=="Y")) & ((d["measurement_type"]=="smoking") & (d["choice"]=="Y"))
         
# Correct 
def is_healthy_smoker(d): 
    """ """
    return [(d["measurement_type"]=="healthy") & (d["choice"]=="Y"), (d["measurement_type"]=="smoking") & (d["choice"]=="Y")]

   
# Wrong 
def not_smoker_y(d):
    """ Be care this might not do what you expect. Excluding a specific characteristica will not eliminate any subject unless it is the only characteristica.
    """
    return ~((d["measurement_type"]=="smoking") & (d["choice"]=="Y")) 
not_smoker_y_data = test_data.subject_filter(not_smoker_y)

#Correct
# exlcude smoker
def smoker_y(d):
    return (d["measurement_type"]=="smoking") & (d["choice"]=="Y")
healthy_data = test_data.subject_exclude(smoker_y)


# Wrong 
def not_disease(d):
    """ Be care this might not do what you expect. Excluding a specific characteristica will not eliminate any subject unless it is the only characteristica
    """
    return  ~(d["measurement_type"]=="disease")
healthy_data = test_data.subject_filter(not_disease)

# Correct 
# exlcude the disease
def disease(d):
    return  d["measurement_type"]=="disease"
healthy_data = test_data.subject_exclude(disease)


Concise DataFrames
Concise DataFrames
Concise DataFrames
Concise DataFrames
